In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.offline import plot
from plotly.subplots import make_subplots

In [2]:
wind = pd.read_csv(r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\final_csv\wind_50hz_gen_2021.csv")
solar = pd.read_csv(r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\final_csv\solar_50hz_gen_2021.csv", low_memory=False)

In [3]:
wind = wind.iloc[:, 1:]
wind.rename(columns={"Strommenge": "generation_kWh", "capacity": "capacity_kW", "station_no": "station"}, inplace=True)

In [4]:
offshore = pd.read_csv(r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\offshore_wind.csv")

wind_offshore = wind[wind.EEG_Mastr_Nr.isin(offshore.EegMaStRNummer)]

wind_off = wind_offshore.copy()

wind_off["gen_station_weight"] = np.where(wind_off.station == 5, wind_off.generation_kWh/(wind_off.generation_kWh[wind_off.station == 5].sum()),
                                            wind_off.generation_kWh/(wind_off.generation_kWh[wind_off.station == 7].sum()))
wind_off["gen_weight"] = np.where(wind_off.station == 5, wind_off.generation_kWh/(wind_off.generation_kWh.sum()),
                                            wind_off.generation_kWh/(wind_off.generation_kWh.sum()))
wind_off["cap_station_weight"] = np.where(wind_off.station == 5, wind_off.capacity_kW/(wind_off.capacity_kW[wind_off.station == 5].sum()),
                                            wind_off.capacity_kW/(wind_off.capacity_kW[wind_off.station == 7].sum()))
wind_off["cap_weight"] = np.where(wind_off.station == 5, wind_off.capacity_kW/(wind_off.capacity_kW.sum()),
                                            wind_off.capacity_kW/(wind_off.capacity_kW.sum()))

In [5]:
wind_off_stations = wind_off.groupby("station").agg({"capacity_kW": "sum", "generation_kWh": "sum", "cap_weight": "sum", "cap_station_weight": "sum", "gen_weight": "sum", "gen_station_weight": "sum"})
wind_off_stations.to_csv("weights_offshore.csv")

In [6]:
px.defaults.color_continuous_scale = px.colors.sequential.Plasma
fig = px.scatter_mapbox(wind_off, lat="lat", lon="lon", hover_name="station", hover_data=["capacity_kW", "generation_kWh"], color="station",
                        color_discrete_sequence=px.colors.qualitative.Light24, zoom=3, height=600, size="gen_weight")
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [7]:
wind = wind[wind.EEG_Mastr_Nr.isin(offshore.EegMaStRNummer)==False]

In [8]:
# defining weight per unit in ralation to total installed capacity
wind["capacity_weight"] = (wind.capacity_kW)/(wind.capacity_kW.sum())

In [9]:
# defining weigth per unit in relation to total realized generation
wind["generation_weight"] = (wind.generation_kWh)/(wind.generation_kWh.sum())

In [10]:
wind["capacity_weight_in_cluster"] =    np.where(wind.station == 0, wind.capacity_kW/(wind.capacity_kW[wind.station == 0].sum()),
                                        np.where(wind.station == 1, wind.capacity_kW/(wind.capacity_kW[wind.station == 1].sum()),
                                        np.where(wind.station == 2, wind.capacity_kW/(wind.capacity_kW[wind.station == 2].sum()),
                                        np.where(wind.station == 3, wind.capacity_kW/(wind.capacity_kW[wind.station == 3].sum()),
                                        np.where(wind.station == 4, wind.capacity_kW/(wind.capacity_kW[wind.station == 4].sum()),
                                        np.where(wind.station == 5, wind.capacity_kW/(wind.capacity_kW[wind.station == 5].sum()),
                                        np.where(wind.station == 6, wind.capacity_kW/(wind.capacity_kW[wind.station == 6].sum()),
                                        wind.capacity_kW/(wind.capacity_kW[wind.station == 7].sum())
                                        )))))))

In [11]:
wind["generation_weight_in_cluster"] =  np.where(wind.station == 0, wind.generation_kWh/(wind.generation_kWh[wind.station == 0].sum()),
                                        np.where(wind.station == 1, wind.generation_kWh/(wind.generation_kWh[wind.station == 1].sum()),
                                        np.where(wind.station == 2, wind.generation_kWh/(wind.generation_kWh[wind.station == 2].sum()),
                                        np.where(wind.station == 3, wind.generation_kWh/(wind.generation_kWh[wind.station == 3].sum()),
                                        np.where(wind.station == 4, wind.generation_kWh/(wind.generation_kWh[wind.station == 4].sum()),
                                        np.where(wind.station == 5, wind.generation_kWh/(wind.generation_kWh[wind.station == 5].sum()),
                                        np.where(wind.station == 6, wind.generation_kWh/(wind.generation_kWh[wind.station == 6].sum()),
                                        wind.generation_kWh/(wind.generation_kWh[wind.station == 7].sum())
                                        )))))))

In [12]:
wind

,station,distance,lat,lon,EEG_Anlagenschlüssel,EEG_Mastr_Nr,PLZ,Ort,Bundesland,Inbetriebnahme,Außerbetriebnahme,Netzabgang,capacity_kW,generation_kWh,capacity_weight,generation_weight,capacity_weight_in_cluster,generation_weight_in_cluster
0,4,25.408486,52.578180,12.872044,E40450010225600000000007819670001,NaN,14641,Markee,Brandenburg,2008-09-09,NaN,2021-12-31,2300.0,3146024.0,1.193901e-04,1.036187e-04,8.600635e-04,7.883141e-04
1,7,50.271753,53.293714,14.011771,E40450010000000000000021907000185,NaN,17291,Grünow,Brandenburg,2017-09-15,NaN,NaN,3450.0,7950098.0,1.790852e-04,2.618477e-04,1.299634e-03,1.707271e-03
2,7,39.418417,53.396101,13.959544,E40450010000000000000082140900138,NaN,17291,Tornow,Brandenburg,2009-08-06,NaN,NaN,2000.0,3205920.0,1.038175e-04,1.055915e-04,7.534111e-04,6.884663e-04
3,7,38.541898,53.405539,14.191801,E40450010000000000000082067000092,NaN,17326,Brüssow,Brandenburg,2008-03-14,NaN,NaN,2000.0,4127208.0,1.038175e-04,1.359354e-04,7.534111e-04,8.863114e-04
4,7,38.541898,53.405539,14.191801,E40450010000000000000082062000085,NaN,17326,Brüssow,Brandenburg,2007-03-22,NaN,NaN,2000.0,3973421.0,1.038175e-04,1.308702e-04,7.534111e-04,8.532859e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10569,2,57.471965,50.901314,10.565306,E4050001S117809759810000000000001,EEG949784572849,99880,Waltershausen,Thüringen,2012-10-12,NaN,NaN,3.5,610.0,1.816806e-07,2.009121e-08,1.280654e-06,1.428011e-07
10570,2,98.122527,51.584880,11.695680,E41081010000000000000000000005382,EEG998985075338,06198,Salzatal,Sachsen-Anhalt,1999-12-19,NaN,NaN,1500.0,2204562.0,7.786313e-05,7.261036e-05,5.488518e-04,5.160883e-04
10571,2,96.821367,51.572017,11.680725,E41081010000000000000000000005386,EEG918004321291,06198,Salzatal,Sachsen-Anhalt,1999-12-22,NaN,NaN,1500.0,2204568.0,7.786313e-05,7.261056e-05,5.488518e-04,5.160897e-04
10572,0,18.649854,50.944946,12.765429,E41081010000000000000000000218962,EEG905063090923,09328,Lunzenau,Sachsen,2014-03-28,NaN,NaN,1.2,15.0,6.229050e-08,4.940461e-10,5.120793e-07,4.073564e-09


In [13]:
# calculationg the weight of assigned weather stations

weighted_stations_wind = wind.groupby("station").agg({"capacity_kW": "sum", "capacity_weight": "sum", "capacity_weight_in_cluster": "sum", "generation_kWh": "sum", "generation_weight": "sum", "generation_weight_in_cluster": "sum"})

weighted_stations_wind

,capacity_kW,capacity_weight,capacity_weight_in_cluster,generation_kWh,generation_weight,generation_weight_in_cluster
station,,,,,,
0,2343386.960,0.121642,1.0,3.682279e+09,0.121281,1.0
1,231115.000,0.011997,1.0,3.943339e+08,0.012988,1.0
2,2732978.049,0.141865,1.0,4.271676e+09,0.140694,1.0
3,2968767.700,0.154105,1.0,4.428748e+09,0.145867,1.0
4,2674221.100,0.138815,1.0,3.990825e+09,0.131443,1.0
5,1425806.600,0.074012,1.0,2.628418e+09,0.086571,1.0
6,4233705.000,0.219766,1.0,6.308644e+09,0.207784,1.0
7,2654593.300,0.137797,1.0,4.656611e+09,0.153372,1.0


In [14]:
#px.defaults.color_continuous_scale = px.colors.sequential.Plasma
#fig = px.scatter_mapbox(wind, lat="lat", lon="lon", hover_name="station", hover_data=["capacity_kW", "generation_kWh"], color="station",
#                        color_discrete_sequence=px.colors.qualitative.Light24, zoom=3, height=600, size="generation_weight")
#fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()

In [15]:
solar_outside = solar[(solar.lon < 9.5) | (solar.lat < 50.1) | (solar.lat > 54.7)]
solar = solar[solar.index.isin(solar_outside.index)==False].reset_index()
solar = solar.iloc[:, 2:]
solar.rename(columns={"Strommenge": "generation_kWh", "capacity": "capacity_kW"}, inplace=True)

In [16]:
solar

,station,distance,lat,lon,EEG_Anlagenschlüssel,EEG_Mastr_Nr,PLZ,Ort,Bundesland,capacity_kW,generation_kWh
0,7,34.510722,53.713084,13.547886,E4186001S000000000000117200200001,EEG920816041762,17392,Boldekow,Mecklenburg-Vorpommern,350.73,244830.0
1,3,46.283741,51.866800,13.731600,E4043001Sg00000000000002496400001,EEG926539762953,15926,Luckau,Brandenburg,528.00,497528.0
2,6,30.930478,52.613461,11.757925,E4186001S000000000000106754100001,EEG945784101287,14656,Brieselang,Brandenburg,100.00,114091.0
3,2,6.828936,51.421972,10.225318,E4183201RS50446719000096741500001,EEG998864951579,37308,Steinbach,Thüringen,678.00,639936.0
4,7,58.317932,54.072290,13.374788,E414560175039001070070MSBS0000001,EEG971272328508,17489,Greifswald,Mecklenburg-Vorpommern,297.00,265792.0
...,...,...,...,...,...,...,...,...,...,...,...
214205,5,52.818568,53.917941,11.406316,E4170301239680010000470036000S001,NaN,23968,Wismar,Mecklenburg-Vorpommern,12.80,4906.0
214206,5,47.783570,53.929729,11.486062,E41703012397000000000000091005252,NaN,23970,Wismar,Mecklenburg-Vorpommern,7.70,4503.0
214207,2,74.827840,51.886670,11.049562,E41180010007603882000009892000298,NaN,38820,Halberstadt,Sachsen-Anhalt,4.50,2331.0
214208,3,9.684241,52.172157,14.247106,E41860010000000000602162985400001,NaN,15848,Beeskow,Brandenburg,11.04,4201.0


In [17]:
# defining weight per unit in ralation to total installed capacity
solar["capacity_weight"] = (solar.capacity_kW)/(solar.capacity_kW.sum())

In [18]:
# defining weigth per unit in relation to total realized generation
solar["generation_weight"] = (solar.generation_kWh)/(solar.generation_kWh.sum())

In [19]:
solar["capacity_weight_in_cluster"] =       np.where(solar.station == 0, solar.capacity_kW/(solar.capacity_kW[solar.station == 0].sum()),
                                            np.where(solar.station == 1, solar.capacity_kW/(solar.capacity_kW[solar.station == 1].sum()),
                                            np.where(solar.station == 2, solar.capacity_kW/(solar.capacity_kW[solar.station == 2].sum()),
                                            np.where(solar.station == 3, solar.capacity_kW/(solar.capacity_kW[solar.station == 3].sum()),
                                            np.where(solar.station == 4, solar.capacity_kW/(solar.capacity_kW[solar.station == 4].sum()),
                                            np.where(solar.station == 5, solar.capacity_kW/(solar.capacity_kW[solar.station == 5].sum()),
                                            np.where(solar.station == 6, solar.capacity_kW/(solar.capacity_kW[solar.station == 6].sum()),
                                            solar.capacity_kW/(solar.capacity_kW[solar.station == 7].sum())
                                            )))))))

In [20]:
solar["generation_weight_in_cluster"] =     np.where(solar.station == 0, solar.generation_kWh/(solar.generation_kWh[solar.station == 0].sum()),
                                            np.where(solar.station == 1, solar.generation_kWh/(solar.generation_kWh[solar.station == 1].sum()),
                                            np.where(solar.station == 2, solar.generation_kWh/(solar.generation_kWh[solar.station == 2].sum()),
                                            np.where(solar.station == 3, solar.generation_kWh/(solar.generation_kWh[solar.station == 3].sum()),
                                            np.where(solar.station == 4, solar.generation_kWh/(solar.generation_kWh[solar.station == 4].sum()),
                                            np.where(solar.station == 5, solar.generation_kWh/(solar.generation_kWh[solar.station == 5].sum()),
                                            np.where(solar.station == 6, solar.generation_kWh/(solar.generation_kWh[solar.station == 6].sum()),
                                            solar.generation_kWh/(solar.generation_kWh[solar.station == 7].sum())
                                            )))))))

In [21]:
solar

,station,distance,lat,lon,EEG_Anlagenschlüssel,EEG_Mastr_Nr,PLZ,Ort,Bundesland,capacity_kW,generation_kWh,capacity_weight,generation_weight,capacity_weight_in_cluster,generation_weight_in_cluster
0,7,34.510722,53.713084,13.547886,E4186001S000000000000117200200001,EEG920816041762,17392,Boldekow,Mecklenburg-Vorpommern,350.73,244830.0,2.595778e-05,2.021198e-05,0.000368,0.000275
1,3,46.283741,51.866800,13.731600,E4043001Sg00000000000002496400001,EEG926539762953,15926,Luckau,Brandenburg,528.00,497528.0,3.907767e-05,4.107351e-05,0.000226,0.000218
2,6,30.930478,52.613461,11.757925,E4186001S000000000000106754100001,EEG945784101287,14656,Brieselang,Brandenburg,100.00,114091.0,7.401073e-06,9.418802e-06,0.000056,0.000076
3,2,6.828936,51.421972,10.225318,E4183201RS50446719000096741500001,EEG998864951579,37308,Steinbach,Thüringen,678.00,639936.0,5.017928e-05,5.283002e-05,0.000335,0.000345
4,7,58.317932,54.072290,13.374788,E414560175039001070070MSBS0000001,EEG971272328508,17489,Greifswald,Mecklenburg-Vorpommern,297.00,265792.0,2.198119e-05,2.194250e-05,0.000312,0.000298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214205,5,52.818568,53.917941,11.406316,E4170301239680010000470036000S001,NaN,23968,Wismar,Mecklenburg-Vorpommern,12.80,4906.0,9.473374e-07,4.050157e-07,0.000011,0.000006
214206,5,47.783570,53.929729,11.486062,E41703012397000000000000091005252,NaN,23970,Wismar,Mecklenburg-Vorpommern,7.70,4503.0,5.698826e-07,3.717459e-07,0.000006,0.000005
214207,2,74.827840,51.886670,11.049562,E41180010007603882000009892000298,NaN,38820,Halberstadt,Sachsen-Anhalt,4.50,2331.0,3.330483e-07,1.924361e-07,0.000002,0.000001
214208,3,9.684241,52.172157,14.247106,E41860010000000000602162985400001,NaN,15848,Beeskow,Brandenburg,11.04,4201.0,8.170785e-07,3.468143e-07,0.000005,0.000002


In [22]:
neg_gen = solar[solar.generation_kWh < 0]

In [23]:
solar_pos = solar[solar.index.isin(neg_gen.index)==False]

In [24]:
# calculationg the weight of assigned weather stations

weighted_stations_solar = solar.groupby("station").agg({"capacity_kW": "sum", "capacity_weight": "sum", "capacity_weight_in_cluster": "sum", "generation_kWh": "sum", "generation_weight": "sum", "generation_weight_in_cluster": "sum"})

weighted_stations_solar

,capacity_kW,capacity_weight,capacity_weight_in_cluster,generation_kWh,generation_weight,generation_weight_in_cluster
station,,,,,,
0,2977571.185,0.220372,1.0,2.638344e+09,0.217809,1.0
1,163652.348,0.012112,1.0,1.108594e+08,0.009152,1.0
2,2023763.077,0.149780,1.0,1.853995e+09,0.153057,1.0
3,2336450.465,0.172922,1.0,2.283648e+09,0.188527,1.0
4,2063363.085,0.152711,1.0,1.936876e+09,0.159899,1.0
5,1204590.195,0.089153,1.0,8.913505e+08,0.073586,1.0
6,1789442.447,0.132438,1.0,1.506244e+09,0.124348,1.0
7,952721.305,0.070512,1.0,8.917955e+08,0.073622,1.0


In [25]:
#px.defaults.color_continuous_scale = px.colors.sequential.Plasma
#fig = px.scatter_mapbox(solar_pos, lat="lat", lon="lon", hover_name="station", hover_data=["capacity_kW", "generation_kWh"], color="station",
#                        color_discrete_sequence=px.colors.qualitative.Light24, zoom=3, height=600, size="generation_weight")
#fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()

In [26]:
weighted_stations_wind.to_csv("weighted_stations_wind.csv")
weighted_stations_solar.to_csv("weighted_stations_solar.csv")